## Unzip The Raw Data

In [ ]:
!./unzip.sh UCI_HAR_Dataset.zip 2>&1 > /dev/null
!pip install --no-dependencies tflite-model-maker

In [ ]:
import os
import warnings

import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1L2

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

warnings.filterwarnings("ignore")
warnings.filterwarnings(
    "ignore", message="calling BaseResourceVariable.__init__.*constraint is deprecated"
)
print(tf.__version__)


## Building The Dataset

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape) + [nb_classes])


def load_y(subset):
    # Get the path
    path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/y_{subset}.txt"

    # Read the file
    y = np.loadtxt(path, delimiter=",", dtype=int)

    # # One-hot encode labels
    one_hot_labels = get_one_hot(y - 1, len(np.unique(y)))
    if subset == "train":
        assert one_hot_labels.shape == (
            7352,
            6,
        ), f"Wrong dimensions: {one_hot_labels.shape} should be (7352, 6)"
    if subset == "test":
        assert one_hot_labels.shape == (
            2947,
            6,
        ), f"Wrong dimensions: {one_hot_labels.shape} should be (2947, 6)"
    assert (
        y[0] - 1 == np.where(one_hot_labels[0] == np.max(one_hot_labels[0]))[0][0]
    ), f"Value mismatch {np.max(one_hot_labels[0])[0][0]} vs {y[13] - 1}"
    return one_hot_labels


def build_data(subset):
    if subset not in ["train", "val", "test"]:
        raise Exception(f"Invalid subset: {subset}")

    folder_path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/"

    # Get all signal files in folder
    signal_files = glob.glob(os.path.join(folder_path, "*.txt"))
    # print(signal_files)

    assert len(signal_files) == 9, f"No signal files found in {folder_path}"
    np.loadtxt(signal_files[0]).shape
    # print(f"{signal_shape}")

    # Determine signal order based on file names
    signal_order = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_",
    ]

    # file_prefix = "UCI_HAR_Dataset/UCI_HAR_Dataset/train/Inertial Signals/"
    # file_suffix = ".txt"
    signal_files = [
        f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/{x}{subset}.txt"
        for x in signal_order
    ]

    # Load signal data from each file and append to signals_data list
    signals_data = [np.loadtxt(x) for x in signal_files]

    # Transpose signal data array so that shape is (number of samples, number of timesteps, number of signals)
    signals_data = np.transpose(signals_data, (1, 2, 0))

    # Verify final shape of combined data
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    if subset == "train":
        assert signals_data.shape == (7352, 128, len(signal_files))
    else:
        assert signals_data.shape == (2947, 128, len(signal_files))
    return signals_data


def load_data():
    return build_data("train"), load_y("train"), build_data("test"), load_y("test")


# Loading the train and test data
X_train, y_train, X_test, y_test = load_data()


In [ ]:
first_sample = X_train[0]
first_timestep = first_sample[0]
assert len(first_sample) == 128
assert first_timestep[0] == 1.8085150e-004, print(first_timestep[0])
assert first_timestep[1] == 1.0766810e-002, print(first_timestep[1])


In [ ]:
# Debug
assert X_train.shape == (7352, 128, 9), print(
    "Expected shape: (7352, 128, 9) get", X_train.shape
)
assert X_test.shape == (2947, 128, 9), print(
    "Expected: (2947, 128, 9) get", X_test.shape
)
assert y_train.shape == (7352, 6), print("Expected: (7352, 6) get", y_train.shape)
assert y_test.shape == (2947, 6), print("Expected: (2947, 6) get", y_test.shape)
assert len(X_train[0][0]) == 9, print("Signals numbers not match")


In [ ]:
import os
import pickle

# Create the "assets" folder if it does not exist
if not os.path.exists("assets"):
    os.mkdir("assets")

# Create the "assets/data" folder if it does not exist
data_folder = os.path.join("assets", "data")
if not os.path.exists(data_folder):
    os.mkdir(data_folder)


def save_data_to_pickle_shards(data, data_name, data_folder):
    # Check if the data already exists
    filename = os.path.join(data_folder, f"{data_name}_0.pickle")
    if os.path.exists(filename):
        print(f"{data_name} already exists in {data_folder}. Skipping data saving.")
        return

    if not os.path.exists(os.path.join(data_folder)):
        os.makedirs(os.path.join(data_folder))

    # Serialize your data
    serialized_data = pickle.dumps(data)

    # Split the serialized data into smaller chunks
    chunk_size = 50 * 1024 * 1024  # 50 megabytes
    chunks = [
        serialized_data[i : i + chunk_size]
        for i in range(0, len(serialized_data), chunk_size)
    ]

    # Save each chunk to a file in the "asset/data" folder
    for i, chunk in enumerate(chunks):
        filename = os.path.join(data_folder, f"{data_name}_{i}.pickle")
        with open(filename, "wb") as f:
            f.write(chunk)


In [ ]:
save_data_to_pickle_shards(X_train, "X_train", data_folder)
save_data_to_pickle_shards(y_train, "y_train", data_folder)
save_data_to_pickle_shards(X_test, "X_test", data_folder)
save_data_to_pickle_shards(y_test, "y_test", data_folder)


In [ ]:
import os
import pickle


def load_data_from_pickle_shards(data_name, data_folder):
    # Find all pickle files that match the data name
    files = sorted(
        [
            os.path.join(data_folder, f)
            for f in os.listdir(data_folder)
            if f.startswith(data_name)
        ]
    )

    # Load the data from each file
    data = b""
    for filename in files:
        with open(filename, "rb") as f:
            data += f.read()

    # Deserialize the data
    return pickle.loads(data)


# Load the data from the pickle shards
loaded_X_train = load_data_from_pickle_shards("X_train", data_folder)
loaded_y_train = load_data_from_pickle_shards("y_train", data_folder)
loaded_X_test = load_data_from_pickle_shards("X_test", data_folder)
loaded_y_test = load_data_from_pickle_shards("y_test", data_folder)


In [ ]:
# Check if the loaded data matches the original data
assert loaded_X_train.shape == X_train.shape
assert loaded_y_train.shape == y_train.shape
assert loaded_X_test.shape == X_test.shape
assert loaded_y_test.shape == y_test.shape

assert (loaded_X_train == X_train).all()
assert (loaded_y_train == y_train).all()
assert (loaded_X_test == X_test).all()
assert (loaded_y_test == y_test).all()


In [ ]:
# function to count the number of classes
def count_classes(y):
    return len(set([tuple(category) for category in y]))


timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = count_classes(y_train)

# Initializing parameters
n_epochs = 30
n_batch = 16

# Bias regularizer value - we will use elasticnet
regularizer = L1L2(0.01, 0.01)

print(f"Timesteps: {timesteps}")
print(f"Input dimention: {input_dim}")
print(f"Total samples: {len(X_train)}")


In [ ]:
# Model execution
model = Sequential()
model.add(
    LSTM(
        64,
        input_shape=(timesteps, input_dim),
        return_sequences=True,
        bias_regularizer=regularizer,
    )
)
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation="sigmoid"))
model.summary()


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# Training the model
model.fit(
    X_train,
    y_train,
    batch_size=n_batch,
    validation_data=(X_test, y_test),
    epochs=n_epochs,
)
model.save("assets/model-backup.h5")


## Verifying Rebuilt Database

In [ ]:
import numpy as np

success_thresh = 10
passed = 0

def compare_array_halves(array1, array2):
    n = len(array1) // 2
    return np.array_equal(array1[:n], array2[n:])


for i in range(len(loaded_X_train)):
    for j in range(len(loaded_X_train)):
        if i == j:
            continue
        result = compare_array_halves(loaded_X_train[i], loaded_X_train[j])
        if result:
            passed += 1
            print(
                f"The first half of array{i} is equal to the second half of array{j}."
            )
            break
    if passed == success_thresh:
        print(f"Success threshhold passed, stopping check")
        break


## Rebuilding Model From Backup

In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("assets/model-backup.h5")


### Checking The Rebuilt Model From Backup File

In [ ]:
test_input = X_test[0].reshape((1, 128, 9))

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)


In [ ]:
assert np.array(model.predict(test_input)).argmax() + 1 == np.array(y_test[0]).argmax() + 1


In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("assets/model-backup.h5")


## Converting Keras Model to TFLite Model

In [ ]:
# TODO